# Таблицы

## Подгтовка

Директория для хранения файлов внеших таблиц:

In [ ]:
HOST=${IMPALA:-impala} execute \
mkdir -p /user/hive/warehouse/extenal_data

База данных для работы:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
DROP DATABASE IF EXISTS tables_demo
CASCADE
"

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
CREATE DATABASE tables_demo
"

## Создание таблиц

### Создание простой таблицы

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d tables_demo -q "
CREATE TABLE my_first_table (
    id INT,
    name STRING
)
"

#### Описание таблицы

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d tables_demo -q "
DESCRIBE FORMATTED my_first_table
"

Файлы таблицы находятся на диске, но сейчас таблица пустая, поэтому директория таблицы пуста:

In [ ]:
HOST=${IMPALA_HOST} execute \
find /user/hive/warehouse/managed/tables_demo.db

Если записать информацию в таблицу:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d tables_demo -q "
INSERT INTO my_first_table
VALUES(1, 'Hello')
"

то можно увидеть, что появились файлы с данными:

In [ ]:
HOST=${IMPALA_HOST} execute \
find /user/hive/warehouse/managed/tables_demo.db

Добавление новых данных:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d tables_demo -q "
INSERT INTO my_first_table
VALUES(2, 'Привет')
"

In [ ]:
HOST=${IMPALA_HOST} execute \
find /user/hive/warehouse/managed/tables_demo.db

> Добавление новых данных создает новые файлы

#### Вывод

1. Каждая `INSERT` операция создает новые файлы;
1. Много небольших файлов плохо сказывается на работе `HDFS`;
1. Желательно записывать сразу много строк за раз, чтобы число файлов было небольшим.

### Сегментированные (partition) таблицы

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d tables_demo -q "
CREATE TABLE movies (
    id INT,
    name STRING,
    release_date DATE
)
PARTITIONED BY (year INT, month INT)
"

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d tables_demo -q "
DESCRIBE EXTENDED movies
"

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d tables_demo -q "
INSERT INTO movies (id, name, release_date, year, month)
WITH records AS(
    SELECT 1 id, 'Titanic' name, DATE '1997-12-19' release_date UNION
    SELECT 2 id, 'Good Will Hunting' name, DATE '1997-12-5' release_date UNION
    SELECT 3 id, 'Starship Troopers' name, DATE '1997-11-7' release_date UNION
    SELECT 4 id, 'L.A. Confidential' name, DATE '1997-9-19' release_date UNION
    SELECT 5 id, 'Gattaca' name, DATE '1997-10-24' release_date UNION
    SELECT 6 id, 'Interstellar' name, DATE '2014-10-26' release_date
)
SELECT id
     , name
     , release_date
     , YEAR(release_date) year
     , MONTH(release_date) month
  FROM records
"

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d tables_demo -q "
SHOW PARTITIONS movies
"

In [ ]:
HOST=${IMPALA_HOST} execute \
find /user/hive/warehouse/managed/tables_demo.db/movies

#### Партиционирование ускоряет запросы

Если к таблице выполняются запросы по определенным полям, то имеет смысл использовать эти поля в качестве ключей сегментирования.

Например, запрос к сегментированой таблице по полю, которое не входит в ключ сегментирования (партиционирования) запускает полное сканирование всех файлов:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d tables_demo -q "
EXPLAIN
SELECT *
  FROM movies
 WHERE name = 'Interstellar'
"

Запрос к сегментированой таблице по полю, которое входит в ключ сегментирования (партиционирования), запускает сканирование только части файлов:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d tables_demo -q "
EXPLAIN
SELECT *
  FROM movies
 WHERE year = 1997
"

Можно еще сильнее сузить область сканирования, если воспользоваться второй частью ключа партиционирования: 

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d tables_demo -q "
EXPLAIN
SELECT *
  FROM movies
 WHERE year = 1997
   AND month = 12
"

Если сегментов по ключу партиционирования нет, то сканирование файловой системы полностью пропускается:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d tables_demo -q "
EXPLAIN
SELECT *
  FROM movies
 WHERE year = -1
"

При этом поиск данных, которых нет, по полям, которые не входят в ключ партиционирования, вынуждает сканировать все файлы:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d tables_demo -q "
EXPLAIN
SELECT *
  FROM movies
 WHERE name = 'Interstellar'
"

Ислючение файлов из сканирования также возможно, если выполнять запрос по любому полю из ключа сегментирования:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d tables_demo -q "
EXPLAIN
SELECT *
  FROM movies
 WHERE month = 10
"

#### Вывод

1. Сегментирование является одним из самых эффективных способов повышения производительности запроса за счёт снижения числа сканируемых файлов;
1. Ключ сегментирования должен быть частью наиболее частых запросов, выполняющихся к таблице по ключу;
1. Сегментирование может привести к проблеме небольших файлов;
1. При сомнениях необходимо отказаться от сегментирования.

### Формат данных таблицы

Файлы таблицы по умолчанию сохраняются в формате parquet, но можно выбрать свой формат.

Например, следующая таблица сохраняет данные в формате `*.txt`:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d tables_demo -q "
CREATE TABLE best_songs (
    id INT,
    name STRING
)
PARTITIONED BY (country STRING, year INT, month INT)
STORED AS TEXTFILE
"

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d tables_demo -q "
INSERT INTO best_songs (id, name, country, year, month)
WITH records (id, name, country, year, month) AS (
    SELECT 1, 'Easy on Me', 'US', 2022, 1 UNION
    SELECT 2, 'We Dont Talk About Bruno', 'US', 2022, 2 UNION
    SELECT 3, 'Heat Waves', 'US', 2022, 3 UNION
    SELECT 4, 'As It Was', 'US', 2022, 4 UNION
    SELECT 5, 'First Class', 'US', 2022, 5
)
SELECT * FROM records
"

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d tables_demo -q "
SELECT *
  FROM best_songs
"

In [ ]:
HOST=${IMPALA_HOST} execute \
find /user/hive/warehouse/managed/tables_demo.db/best_songs

In [ ]:
file_name=$(HOST=${IMPALA_HOST:-impala} execute find /user/hive/warehouse/managed/tables_demo.db/best_songs/country=US/year=2022/month=2 -name '*.txt')
HOST=${IMPALA_HOST:-impala} execute \
cat "${file_name}"

#### Вывод

1. Impala позволяет выбрать наиболее приемлемый формат файлов для хранения данных;
1. Колоночные форматы файлов (Parquet, ORC) больше подходят для аналитических запросов;
1. Строчные форматы файлов (AVRO, TXT) подходят для запросов связанных с разбиением результирующего множества на страницы.

### Клонирование таблицы

Impala позволяет создать таблицу на основе другой таблицы при помощи `CREATE TABLE ... AS SELECT` (`CTAS`):

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d tables_demo -q "
CREATE TABLE my_top_songs
AS
    SELECT id
         , name
         , year
         , month
      FROM best_songs
"

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d tables_demo -q "
DESCRIBE EXTENDED my_top_songs
"

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d tables_demo -q "
SELECT *
  FROM my_top_songs
"

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d tables_demo -q "
CREATE TABLE my_top_songs_part
    PARTITIONED BY (year, month)
    COMMENT 'clone of best_songs'
    STORED AS PARQUET
AS
    SELECT id
         , name
         , year
         , month
      FROM best_songs
"

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d tables_demo -q "
SHOW PARTITIONS my_top_songs_part
"

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d tables_demo -q "
SELECT *
  FROM my_top_songs_part
"

#### Вывод

1. `CTAS` копирует только поля, их типы и данные из таблицы;
1. Дополнительную конфигурацию необходимо указывать явно.

### Статистика

Если посмотреть план запроса, то можно заметить, что Impala предупреждает, что для таблицы отсутствует статистика:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d tables_demo -q "
EXPLAIN
SELECT *
  FROM my_top_songs_part
"

Для корректной работы оптимизатора необходимо регулярно собирать статистику:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d tables_demo -q "
COMPUTE STATS my_top_songs_part
"

После сбора статистики Impala не предупреждает о проблемах со статистикой:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d tables_demo -q "
EXPLAIN
SELECT *
  FROM my_top_songs_part
"

#### Вывод

1. Оптимизатор запросов активно использует статистику для выбора наилучшего плана запроса;
1. Чем более свежая статистика доступна, тем точнее оптимизатор может заранее оценить стоимость запроса;
1. Устаревшая статистика или ее отсутствие не влияет на полноту результирующего множества;
1. Можно использовать cron-задачу для сбора статистики на регулярной основе.

### Внешние таблицы

In [ ]:
HOST=${IMPALA_HOST} \
new_file /tmp/external.csv <<EOF
1,Hello
2,Привет
3,Bonjour
EOF

HOST=${IMPALA_HOST} execute \
cp /tmp/external.csv /user/hive/warehouse/extenal_data/greetings.csv

HOST=${IMPALA_HOST} execute \
cat /user/hive/warehouse/extenal_data/greetings.csv

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d tables_demo -q "
CREATE EXTERNAL TABLE greeting_external_table
(
    id INT,
    name STRING
)
COMMENT 'External table for greeting on different languages'
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
STORED AS TEXTFILE
LOCATION '/user/hive/warehouse/extenal_data/';
"

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d tables_demo -q "
SELECT *
  FROM greeting_external_table
"

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d tables_demo -q "
DESCRIBE FORMATTED greeting_external_table
"

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d tables_demo -q "
INSERT INTO greeting_external_table
VALUES(4, 'Hallo')
"

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d tables_demo -q "
SELECT *
  FROM greeting_external_table
"

In [ ]:
HOST=${IMPALA_HOST} execute \
ls /user/hive/warehouse/extenal_data

In [ ]:
file_path=$(HOST=${IMPALA_HOST} execute find /user/hive/warehouse/extenal_data -name '*.txt')

HOST=${IMPALA_HOST} execute \
cat ${file_path}

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d tables_demo -q "
DROP TABLE greeting_external_table
"

In [ ]:
HOST=${IMPALA_HOST} execute \
ls /user/hive/warehouse/extenal_data/

#### Вывод

1. При помощи внешних таблиц можно загружать данные в Impala;
1. Внешние таблицы поддерживают вставку данных, но при этом создается новый файл;
1. Удаление внешней таблицы не удаляет файлы, из которых внешняя таблица загружает данные.

## Вывод

1. Таблицы в Impala являются обёртками над файлами;
1. Таблицы могут хранить данные в разных форматах: Parquet, AVRO, TEXT, и т.д.;
1. Таблицы могут быть партиционированными;
1. Партиционирование значительно повышает производительность запросов, но при этом партиционированные таблицы могут эффективно работать с определенными видами запросов;
1. Impala предлагает традиционный для реляционных баз данных способ клонирования таблицы через `CTAS`: `CREATE TABLE AS SELECT`;
1. Статистика сильно помогает оптимизатору при выборе оптимального плана запроса;
1. Загрузка данных в Impala возможна через внешние таблицы.

### Задания

1. Создать таблицу `movies_kinopoisk`, сегментированную по жанру, со списком 10 самых популярных фильмов сайта [kinopoisk](https://www.kinopoisk.ru/lists/movies/top250/). Сохранять данные в формате txt;
1. Убедиться, что запросы по жанру к таблице `movies_kinopoisk` сканируют только нужные файлы;
1. Собрать статистику по таблице `movies_kinopoisk`;
1. Создать клон таблицы `movies_kinopoisk` под названием `movies_kinopoisk_by_year`, которая сохраняет данные в формате parquet и сегментирована по году выпуска фильма;
1. Убедиться, что запросы к таблице `movies_kinopoisk_by_year` по году сканируют только нужные файлы;
1. Скопировать один из файлов `movies_kinopoisk_by_year` в `/user/hive/warehouse/movie_data` в impala Docker сервис;
1. Создать внешнюю таблицу из файла в директории `/user/hive/warehouse/movie_data`.